In [ ]:
import tftables
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from vae import VariationalAutoencoder
from conv_vae import ConvolutionalVAE
from data import toy_data
from data import data_loader

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
sess = tf.compat.v1.InteractiveSession()
tf.compat.v1.disable_eager_execution()

In [ ]:
input_size = 2**15 #1569128
latent_rep_size = 40
label_size = 2
n_data = 4000 # if using hdf5 file, should be set to match file
learning_rate = .0005
batch_size = 50

network_architecture = {
    'input_size': input_size,
    'latent_representation_size': latent_rep_size,
    'encoder_layer_sizes': [input_size, 100, 100, 90, 90, 80, 80, 2 * latent_rep_size],
    'decoder_layer_sizes': [latent_rep_size, 80, 80, 90, 90, 100, 100, input_size],
    'label_predictor_layer_sizes': [latent_rep_size, 40, 20, 20, 10, 10, 5, 5, label_size]
}


In [ ]:
data_source = 'toy'

In [ ]:
if data_source == 'toy':
    spectra, labels = toy_data.generate_spectra(n_data, network_architecture['input_size'])
    labels = labels[:, 1:3] # ignore temperature and sigma (leaving A, mu)
    input_stream, label_stream, initialize_stream = (
        data_loader.create_loader_from_array(sess, batch_size, spectra, labels)
    )
elif data_source == 'real':
    input_stream, label_stream, initialize_stream = (
        data_loader.create_loader_from_hdf5(sess, batch_size, 'data/sample_real_stars.h5')
    )

In [ ]:
vae = VariationalAutoencoder(sess, network_architecture, input_stream, label_stream, learning_rate, batch_size)

In [ ]:
total_batches = int(n_data / batch_size)

costs = []
l1_costs = []
l2_costs = []
l_costs = []
label_costs = []

In [ ]:
training_epochs = 5

for epoch in tqdm(range(1, training_epochs + 1), desc='Epochs'):
    initialize_stream()
    with tqdm(total=total_batches) as pbar:
        for batch in range(total_batches):
            pbar.set_description('Epoch ' + str(epoch))
            _, cost, l1_loss, r_cost, l_cost, _, m_cost = vae.optimize()

            costs += [cost]
            l1_costs += [l1_loss]
            l2_costs += [r_cost]
            l_costs += [l_cost]
            label_costs += [m_cost]
            
            pbar.set_postfix(loss=cost, l1=l1_loss, r=r_cost, l=l_cost, m=m_cost)
            pbar.update()

In [ ]:
# compare to Epoch 20 Batch 160 Iter 003200 | r_cost=17809.174 l_cost=1558.191 l1_loss=0.032 time=8.9

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(2, 1, 1)
plt.plot(costs)
plt.plot(l2_costs)
plt.plot(l_costs)
plt.yscale('log')

plt.subplot(2, 1, 2)
plt.plot(label_costs)
plt.yscale('log')

In [ ]:
initialize_stream()
spectra = sess.run(input_stream)

In [ ]:
i = np.random.randint(0, batch_size)
spectrum = spectra[i, :, 0]
print(i)

In [ ]:
reconstruction = vae.reconstruct(spectrum)

In [ ]:
window = (0, input_size)
#window = (700000, 700000+2**15)

In [ ]:
plt.figure(figsize=(16, 12))
plt.subplot(2, 1, 1)
plt.plot(spectrum[window[0]:window[1]])
reconstruction[reconstruction == 0] = np.nan # get rid of zeros just for the plot
plt.plot(reconstruction[window[0]:window[1]])

plt.subplot(2, 1, 2)
plt.plot((spectrum - reconstruction)[window[0]:window[1]])

In [ ]:
outfile_name = 'output/output-{}'.format(datetime.now().strftime('%y%m%d-%H%M%S'))
vae.save(outfile_name)

In [ ]:
vae.restore('output/output-191002-000332')

In [ ]:
vae.close()